Pulled from: https://github.com/asquare08/AMM-Models/blob/main/Curve%20AMM%20plots.ipynb

Docs here: https://atulagarwal.dev/posts/curveamm/stableswap/

Also excerpts from the real Curve contracts:
https://etherscan.io/address/0xbebc44782c7db0a1a60cb6fe97d0b483032ff1c7#code
https://github.com/curvefi/curve-contract/blob/master/contracts/pools/3pool/StableSwap3Pool.vy

Useful Colab tips: https://medium.com/@robertbracco1/configuring-google-colab-like-a-pro-d61c253f7573#46ce

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from scipy import optimize

In [4]:
# !pip install vyper==0.2.4
!pip install eth-brownie
# !pip install jsonschema==4.3.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 232 kB 5.1 MB/s 
     |████████████████████████████████| 227 kB 38.8 MB/s 
     |████████████████████████████████| 61 kB 6.7 MB/s 
     |████████████████████████████████| 636 kB 38.2 MB/s 
     |████████████████████████████████| 1.9 MB 32.3 MB/s 
     |████████████████████████████████| 381 kB 39.0 MB/s 
     |████████████████████████████████| 138 kB 44.3 MB/s 
     |████████████████████████████████| 45 kB 2.8 MB/s 
     |████████████████████████████████| 2.0 MB 10.1 MB/s 
     |████████████████████████████████| 481 kB 45.5 MB/s 
     |████████████████████████████████| 280 kB 41.6 MB/s 
     |████████████████████████████████| 103 kB 41.9 MB/s 
     |████████████████████████████████| 1.0 MB 38.7 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
     |████████████████████████████████| 1.4 MB 35.5 MB/s 
     |████████████████████████████████

In [5]:
from brownie import *
p = project.load('curve-contract', name="CurveContractProject")
p.load_config()
from brownie.project.CurveContractProject import *
network.connect('development')

FileNotFoundError: ignored

In [ ]:
N_COINS = 3

def get_y(i, j, x, xp_):
    # x in the input is converted to the same price/precision

    assert i != j       # dev: same coin
    assert j >= 0       # dev: j below zero
    assert j < N_COINS  # dev: j above N_COINS

    # should be unreachable, but good for safety
    assert i >= 0
    assert i < N_COINS

    amp: uint256 = self._A()
    D: uint256 = self.get_D(xp_, amp)
    c: uint256 = D
    S_: uint256 = 0
    Ann: uint256 = amp * N_COINS

    _x: uint256 = 0
    for _i in range(N_COINS):
        if _i == i:
            _x = x
        elif _i != j:
            _x = xp_[_i]
        else:
            continue
        S_ += _x
        c = c * D / (_x * N_COINS)
    c = c * D / (Ann * N_COINS)
    b: uint256 = S_ + D / Ann  # - D
    y_prev: uint256 = 0
    y: uint256 = D
    for _i in range(255):
        y_prev = y
        y = (y*y + c) / (2 * y + b - D)
        # Equality with the precision of 1
        if y > y_prev:
            if y - y_prev <= 1:
                break
        else:
            if y_prev - y <= 1:
                break
    return y

In [3]:
from brownie import *
p = project.load('curve-contract', name="CurveContractProject")
p.load_config()
from brownie.project.CurveContractProject import *
network.connect('development')

ModuleNotFoundError: ignored

In [1]:
import os

os.system('git clone --depth 1 https://github.com/curvefi/curve-contract.git')

0

In [ ]:
delta = 20
xr = np.arange(0.1, 2000.0, delta)
yr = np.arange(0.1, 2000.0, delta)
x, y = np.meshgrid(xr, yr)

N = 2
D = 1000
A = 10

Ann = A*N**N
equation = Ann*(x+y) + D - Ann*D - (D**(N+1))/(N**N*x*y)

fig = go.Figure(data =
    go.Contour(
        x=xr,
        y=yr,
        z=equation
    ))
fig.show()


In [ ]:
delta = 20
xr = np.arange(0.1, 2000.0, delta)
yr = np.arange(0.1, 2000.0, delta)
x, y = np.meshgrid(xr, yr)

N = 2
D = 1000
A = 10

Ann = A*N**N
equation = Ann*(x+y) + D - Ann*D - (D**(N+1))/(N**N*x*y)
# print(equation)
# print(np.where(np.isclose(equation, 0, rtol=50.0, atol=50.0)))
def eq_fn(x, y):
  return Ann*(x+y) + D - Ann*D - (D**(N+1))/(N**N*x*y)
eq_fn_wrapper = lambda l: eq_fn(l[0], l[1])
# guess = np.vstack([x.ravel(), y.ravel()])
# guess = np.array(np.meshgrid(xr, yr)).T.reshape(2, -1)
guess = np.array([x, y])
# guess = np.zeros((yr.shape[0], xr.shape[0]))
print(guess)
PRECISION = 1.48
optimized = optimize.root(eq_fn_wrapper, guess, tol=100)
print(optimized)
# print(np.zeros((xr.shape[0], yr.shape[0])))
# print(optimized)
# df_describe = pd.DataFrame(equation)
# df_describe.describe()

[[1.0000e-01 1.0000e-01 1.0000e-01 ... 1.9601e+03 9.8010e+02 1.9801e+03]
 [1.0001e+03 1.0000e-01 1.0001e+03 ... 1.9601e+03 1.9801e+03 1.9801e+03]]


TypeError: ignored

In [ ]:
delta = 200
xr = np.arange(delta, 2000.0, delta)
yr = np.arange(delta, 2000.0, delta)
zr = np.arange(delta, 2000.0, delta)
x, y, z = np.meshgrid(xr, yr, zr)


N = 3
D = 1000
A = 10

Ann = A*N**N
equation = Ann*(x+y+z) + D - Ann*D - (D**(N+1))/(N**N*x*y*z)

PRECISION = 1.48
# NB: equation is currently a 3D array, needs to be callable
# optimized = optimize.newton(equation, [x, y, z], tol=PRECISION, maxiter=50)

# volume = go.Surface(x=xr, y=yr, z=zr, value=equation)

layout = go.Layout(
    title='Parametric Plot',
    scene=dict(
        xaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        yaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        zaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        )
    )
)

# fig = go.Figure(data=volume, layout=layout)
# fig.show()

### StableSwap Invariant
\begin{equation}
An^n\sum x_i + D = DAn^n + \frac{D^{n+1}}{n^n\prod x_i}
\end{equation}

#### Affect of varying Ann, amplification coefficient, on stable swap invariant

In [ ]:
plt.rcParams["figure.figsize"] = [7.50, 4.50]
plt.rcParams["figure.autolayout"] = True
delta = 2
xrange = np.arange(0.1, 2000.0, delta)
yrange = np.arange(0.1, 2000.0, delta)
x, y = np.meshgrid(xrange, yrange)
N = 2
D = 1000
A = 10

Ann_1 = A*N**N
equation = Ann_1*(x+y) + D - Ann_1*D - (D**(N+1))/(N**N*x*y)
cntr1 = plt.contour(x, y, equation, [0], colors='orange')

Ann_2 = A*N
equation = Ann_2*(x+y) + D - Ann_2*D - (D**(N+1))/(N**N*x*y)
cntr2 = plt.contour(x, y, equation, [0], colors='green')

Ann_3 = A
equation = Ann_3*(x+y) + D - Ann_3*D - (D**(N+1))/(N**N*x*y)
cntr3 = plt.contour(x, y, equation, [0], colors='blue')

h1,_ = cntr1.legend_elements()
h2,_ = cntr2.legend_elements()
h3,_ = cntr3.legend_elements()
plt.legend([h1[0], h2[0], h3[0]], ['Ann = A*N**N = {}'.format(Ann_1), 'Ann = A*N = {}'.format(Ann_2), 'Ann = A = {}'.format(Ann_3)])
plt.show()

### CurveCrypto Invariant 
\begin{equation}
KD^{N-1}\sum x_i + \prod x_i = KD^N + \Big(\frac{D}{N}\Big)^N
\end{equation}
where,
\begin{equation}
K_0 = \frac{\prod x_i N^N}{D^N},\hskip{2em} K = AK_0\frac{\gamma^2}{\big(\gamma+1-K_0\big)^2}
\end{equation}

In [ ]:
plt.rcParams["figure.figsize"] = [7.50, 4.5]
plt.rcParams["figure.autolayout"] = True
delta = 2
xrange = np.arange(0.1, 4000.0, delta)
yrange = np.arange(0.1, 4000.0, delta)
x, y = np.meshgrid(xrange, yrange)
N = 2
D = 1000
A = 100
Ann = A*N**N

gamma_1 = 10**(-4)
K0 = x*y*N**N/D**N
K = (Ann*x*y*gamma_1**2)/(D**N*(gamma_1 + 1 - K0)**2)
equation = K*D**(N-1)*(x+y) + x*y - K*D**N - (D/N)**N

cntr1 = plt.contour(x, y, equation, [0], colors='orange')

gamma_2 = 10**(-1)
K0 = x*y*N**N/D**N
K = (A*K0*gamma_2**2)/(gamma_2 + 1 - K0)**2
equation = K*D**(N-1)*(x+y) + x*y - K*D**N - (D/N)**N

cntr2 = plt.contour(x, y, equation, [0], colors='green')

h1,_ = cntr1.legend_elements()
h2,_ = cntr2.legend_elements()
plt.legend([h1[0], h2[0]], ['gamma = {}'.format(gamma_1), 'gamma = {}'.format(gamma_2)])
plt.show()

#### Affect of varying A on CurveCrypto Invariant

In [ ]:
plt.rcParams["figure.figsize"] = [7.50, 4.5]
plt.rcParams["figure.autolayout"] = True
delta = 2
xrange = np.arange(0.1, 4000.0, delta)
yrange = np.arange(0.1, 4000.0, delta)
x, y = np.meshgrid(xrange, yrange)
N = 2
D = 1000
A = 100000000
gamma = 10**(-4)
K0 = x*y*N**N/D**N

Ann_1 = A*N**N
K = (Ann_1*x*y*gamma**2)/(D**N*(gamma + 1 - K0)**2)
equation = K*D**(N-1)*(x+y) + x*y - K*D**N - (D/N)**N
cntr1 = plt.contour(x, y, equation, [0], colors='orange')

Ann_2 = A*N
K = (Ann_2*x*y*gamma**2)/(D**N*(gamma + 1 - K0)**2)
equation = K*D**(N-1)*(x+y) + x*y - K*D**N - (D/N)**N
cntr2 = plt.contour(x, y, equation, [0], colors='green')

Ann_3 = A
K = (Ann_3*x*y*gamma**2)/(D**N*(gamma + 1 - K0)**2)
equation = K*D**(N-1)*(x+y) + x*y - K*D**N - (D/N)**N
cntr3 = plt.contour(x, y, equation, [0], colors='blue')

h1,_ = cntr1.legend_elements()
h2,_ = cntr2.legend_elements()
h3,_ = cntr3.legend_elements()
plt.legend([h1[0], h2[0], h3[0]], ['Ann = A*N**N = {}'.format(Ann_1), 'Ann = A*N = {}'.format(Ann_2), 'Ann = A = {}'.format(Ann_3)])
plt.show()